<a href="https://colab.research.google.com/github/shreyasbkgit/dllab/blob/main/Week_6_Transfer_Learningalexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Week-6: Transfer Learning**

Implement the standard LeNet, AlexNet, VGG CNN architecture model to classify multicategory image dataset.

MNIST handwritten digits (0-9)

Note down accuracies obtained for epochs 5, 50, 250.



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# 1. Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 2. Transform: Resize to 64x64 (safe for AlexNet)
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Prevent shrinking to < 1x1
    transforms.ToTensor()
])

# 3. Datasets and Loaders
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 4. AlexNet model
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),  # 64x64 → 15x15
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # 15x15 → 7x7
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # → 3x3
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),  # → 1x1
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 1 * 1, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# 5. Train & Evaluate Function
def train_and_evaluate(model, epochs=5):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")

    # Evaluation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    acc = 100 * correct / total
    print(f"\nTest Accuracy: {acc:.2f}%")
    return acc

# 6. Run Training
alexnet_acc = train_and_evaluate(AlexNet(), epochs=5)
alexnet_acc = train_and_evaluate(AlexNet(), epochs=50)
alexnet_acc = train_and_evaluate(AlexNet(), epochs=250)

Epoch 1/5: 100%|██████████| 938/938 [00:22<00:00, 42.55it/s]


Epoch 1: Loss = 320.6687


Epoch 2/5: 100%|██████████| 938/938 [00:22<00:00, 42.56it/s]


Epoch 2: Loss = 79.7515


Epoch 3/5: 100%|██████████| 938/938 [00:22<00:00, 42.58it/s]


...

Epoch 21/250: 100%|██████████| 938/938 [00:22<00:00, 41.57it/s]


Epoch 21: Loss = 22.7519


Epoch 22/250: 100%|██████████| 938/938 [00:22<00:00, 41.29it/s]


Epoch 22: Loss = 28.2751


Epoch 23/250:  52%|█████▏    | 488/938 [00:11<00:10, 43.98it/s]